# Gesture Recognition
#### - Kushagra Sengar and Nidhi Mantri
In this group project, we are going to experiment with Conv3d and CNN-RNN architectures that will be able to predict the 5 gestures correctly.

In [ ]:
# Libraries imported
import numpy as np
import os
from skimage import io
import datetime
import os
from skimage.transform import resize


We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.compat.v1.set_random_seed(30)

We have our zip file on drive, mounting drive to get the zip file and extracting it to get the contents of train and validation data.

In [ ]:
##mount the drive to get the dataset
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Project_data.zip", 'r')
zip_ref.extractall("/content/tmp")
zip_ref.close()

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
# Here we are reading train and val csv files
train_doc = np.random.permutation(open('/content/tmp/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/tmp/Project_data/val.csv').readlines())

batch_size =32
# here x is the number of images we are experimenting with, out of 30 (total images in each video)
x=18
# y and z are the height and width of image
y=84
z=84

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. We have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.


img_idx - indices of images (in each folder)

In [ ]:
def generator(source_path, folder_list, batch_size):
    # print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]#create a list of image numbers we want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches =int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #if image.shape == (360,360,3): ###if images are of 360 x 360
                    image = resize(image,(y,z,3))
                   # else:
                        #image = image[:,(image.shape[0] - y ) // 2 : image.shape[0] - (image.shape[0] - y ) // 2] ### if lower quality just centre crop


                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255.0#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255.0#normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        if (len(t)%batch_size)!=0:
          for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((len(t)%batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = io.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #if image.shape == (360,360,3): ###if images are of 360 x 360
                    image = resize(image,(y,z,3))
                    #else:
                        #image = image[:,(image.shape[0] - y ) // 2 : image.shape[0] - (image.shape[0] - y ) // 2] ### if lower quality just centre crop

                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255.0#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255.0#normalise and feed in the image

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do



Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
# note these paths are particular to google colab session
curr_dt_time = datetime.datetime.now()
train_path = '/content/tmp/Project_data/train'
val_path = '/content/tmp/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

###**This is the first model (Conv3d) mentioned in Write-up file**

In [ ]:
# libraries required for model architecture
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

#write our model here
model=Sequential()
# first conv3d block
model.add(Conv3D(32,(3,3,3),strides=(1,1,1),padding="same",input_shape=(18,84,84,3)))
model.add(BatchNormalization())
model.add(Activation("elu"))
model.add(MaxPooling3D(pool_size=(2,2,1),strides=(2,2,1)))
# second conv3d block
model.add(Conv3D(128,(3,3,3),strides=(1,1,1),padding="same"))
model.add(BatchNormalization())
model.add(Activation("elu"))
model.add(MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2)))
# third conv3d block
model.add(Conv3D(256,(3,3,3),strides=(1,1,1),padding="same"))
model.add(BatchNormalization())
model.add(Activation("elu"))
model.add(MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2)))
# let's flatten and use dropout as the regularizer
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512,activation="elu"))
model.add(Dropout(0.5))
model.add(Dense(5,activation="softmax")) # 5 because there are 5 classes


Now that we have written the model, the next step is to `compile` the model. When we print the `summary` of the model, we'll see the total number of parameters we have to train.

In [ ]:
# optimiser and using categorical accuracy as metric
optimiser = tf.keras.optimizers.legacy.SGD(learning_rate=0.001,decay=1e-6,momentum=0.7, nesterov=True)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 18, 84, 84, 32)    2624      
                                                                 
 batch_normalization (Batch  (None, 18, 84, 84, 32)    128       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 18, 84, 84, 32)    0         
                                                                 
 max_pooling3d (MaxPooling3  (None, 9, 42, 84, 32)     0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 9, 42, 84, 128)    110720    
                                                                 
 batch_normalization_1 (Bat  (None, 9, 42, 84, 128)    5

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
# generating training and validation generator to be fed in training
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists('/content/tmp/Model1st/'+model_name):
    os.makedirs('/content/tmp/Model1st/'+model_name, exist_ok=True)

filepath = "/content/tmp/Model1st/"+model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# saving best model and it's weights only
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
# reducing LR based on val loss
LR =ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
# deriving steps per epoch based on number of training videos/folder and batch size
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, we'll be able to save the best model based on val loss (so far).

In [ ]:
#fitting training and validation generator to model
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-13-7f8f9d39394d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 6.4506 - categorical_accuracy: 0.3454
Epoch 1: val_loss improved from inf to 3.62325, saving model to /content/tmp/Model1st/model_init_2023-12-1012_19_00.095192/model-00001-6.45061-0.34540-3.62325-0.18000.h5
21/21 [==============================] - 143s 6s/step - loss: 6.4506 - categorical_accuracy: 0.3454 - val_loss: 3.6233 - val_categorical_accuracy: 0.1800 - lr: 0.0010
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 0.7614 - categorical_accuracy: 0.7405
Epoch 2: val_loss improved from 3.62325 to 2.16544, saving model to /content/tmp/Model1st/model_init_2023-12-1012_19_00.095192/model-00002-0.76136-0.74052-2.16544-0.31000.h5
21/21 [==============================] - 105s 5s/step - loss: 0.7614 - categorical_accuracy: 0.7405 - val_loss: 2.1654 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 3/50
21/21 [==============================] - ETA: 0s - loss: 1.5353 - categorical_accuracy: 0.4853
Epoc

# Note - Code below is repeatative but model architecture is different.. it's just for experimenting purpose.

###**This is the fourth model (VGG - Conv2d - LSTM) mentioned in Write-up file**
The other two architectures are mentioned at the end of notebook

In [ ]:
# Although no need to import all the libraries again and again but
# it's just because we experimented with various sets of models at different times..
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, LSTM, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers import Conv3D, MaxPooling3D
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications.vgg16 import VGG16
import keras

#write our model here
base_model=VGG16(include_top=False,weights='imagenet',input_shape=(84,84,3))
for layers in base_model.layers[:-4]:
  layers.trainable=False

model=Sequential()
model.add(TimeDistributed(base_model,input_shape=(18,84,84,3)))
model.add(TimeDistributed(Conv2D(1024,(3,3),strides=(1,1),padding="same")))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation("elu")))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),strides=(2,2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(512,return_sequences=True))
model.add(LSTM(256))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dense(5,activation='softmax'))




58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
# optimiser and using categorical accuracy as our metric
optimiser = tf.keras.optimizers.legacy.SGD(lr=0.001,decay=1e-6,momentum=0.7, nesterov=True)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 18, 2, 2, 512)     14714688  
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 18, 2, 2, 1024)    4719616   
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 18, 2, 2, 1024)    4096      
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 18, 2, 2, 1024)    0         
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 18, 1, 1, 1024)    0

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# creating training and validation generator to be fed in training
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists("/content/tmp/Conv2DLSTM/"+model_name):
    os.makedirs("/content/tmp/Conv2DLSTM/"+model_name, exist_ok=True)

filepath = "/content/tmp/Conv2DLSTM/"+model_name + 'conv2d-lstm-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
# saving best model and it's weights only
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
# reducing LR based on validation loss
LR =ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
# deriving steps per epoch based on number of training videos/folder and batch size
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
# fitting generators to model
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-13-7f8f9d39394d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 1.6179 - categorical_accuracy: 0.2459
Epoch 1: val_loss improved from inf to 1.57553, saving model to /content/tmp/Conv2DLSTM/model_init_2023-12-1214_38_03.159563/conv2d-lstm-00001-1.61786-0.24585-1.57553-0.32000.h5
21/21 [==============================] - 157s 6s/step - loss: 1.6179 - categorical_accuracy: 0.2459 - val_loss: 1.5755 - val_categorical_accuracy: 0.3200 - lr: 0.0010
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 1.2067 - categorical_accuracy: 0.6906
Epoch 2: val_loss improved from 1.57553 to 1.54259, saving model to /content/tmp/Conv2DLSTM/model_init_2023-12-1214_38_03.159563/conv2d-lstm-00002-1.20667-0.69062-1.54259-0.33000.h5
21/21 [==============================] - 96s 5s/step - loss: 1.2067 - categorical_accuracy: 0.6906 - val_loss: 1.5426 - val_categorical_accuracy: 0.3300 - lr: 0.0010
Epoch 3/50
21/21 [==============================] - ETA: 0s - loss: 1.3578 - categorical_accura

###**This is the second model (VGG - GRU) mentioned in Write-up file**

In [ ]:
#write our model here
base_model=VGG16(include_top=False,weights='imagenet',input_shape=(84,84,3))

for layers in base_model.layers[:-4]:
  layers.trainable=False

model=Sequential()
model.add(TimeDistributed(base_model,input_shape=(18,84,84,3)))
model.add(TimeDistributed(Flatten()))
model.add(GRU(128,return_sequences=True))
model.add(GRU(64))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='softmax'))


###**This is the third model (VGG - Conv2d - GRU) mentioned in Write-up file**

In [ ]:
#write our model here
base_model=VGG16(include_top=False,weights='imagenet',input_shape=(84,84,3))

for layers in base_model.layers[:-4]:
  layers.trainable=False

model=Sequential()
model.add(TimeDistributed(base_model,input_shape=(18,84,84,3)))
model.add(TimeDistributed(Conv2D(128,(3,3),strides=(1,1),padding="same")))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Activation("elu")))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),strides=(2,2))))
model.add(TimeDistributed(Flatten()))
model.add(GRU(128,return_sequences=True))
model.add(GRU(64))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(5,activation='softmax'))